In [2]:
import cv2
import dlib
import numpy as np
import sys
import os

from PIL import Image

import torch
import torch.nn as nn
import torch.nn.parallel
from torch.nn import functional as F
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.autograd import Function, Variable

In [3]:
def faceDetectOpenCV(image, count):
    imageGray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    detector = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
    faces = detector.detectMultiScale(imageGray, 1.1, 2)
    temp = 0
    for x, y, w, h in faces:
        cv2.imwrite("faces/face-" + str(count) + "-" + str(temp) + ".jpg", image[y : y + h, x : x + w])
        temp = temp + 1
        
    for x, y, w, h in faces:    
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
        
    cv2.imwrite("faces/result-" + str(count) + ".jpg", image)

def faceDetectDlib(image, count):
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    detector = dlib.get_frontal_face_detector()
    dets = detector(img, 1)
    for index, face in enumerate(dets):
        left = face.left()
        top = face.top()
        right = face.right()
        bottom = face.bottom()
        cv2.imwrite("faces/face-" + str(count) + "-" + str(index) + ".jpg", image[top : bottom, left : right])
    
    for index, face in enumerate(dets):
        left = face.left()
        top = face.top()
        right = face.right()
        bottom = face.bottom()
        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
        
    cv2.imwrite("faces/result-" + str(count) + ".jpg", image)
    
    
def getFaceFromVideo():
    capture = cv2.VideoCapture("sample.mp4")
    count = 0
    while True:
        ret, frame = capture.read()
        if frame is None:
            break
        elif count % 30 == 0:
            #faceDetectOpenCV(frame, count)
            faceDetectDlib(frame, count)
        count = count + 1
    print("done")

In [40]:
getFaceFromVideo()

done


In [43]:
sampleImage = "faces/face-60-0.jpg"

def getFeaturePoints():
    detector = dlib.get_frontal_face_detector()
    sp = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    bgr_img = cv2.imread(sampleImage)
    rgb_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)
    dets = detector(rgb_img, 1)
    for det in dets:
        landmarks = sp(rgb_img, det)
        
        for index, pt in enumerate(landmarks.parts()):
            pt_pos = (pt.x, pt.y)
            cv2.circle(bgr_img, pt_pos, 2, (255, 0, 0), 1)
        
        cv2.imwrite("landmarks/sampleImage.jpg", bgr_img)

In [44]:
getFeaturePoints()

In [80]:
def getAlignment():
    detector = dlib.get_frontal_face_detector()
    sp = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    bgr_img = cv2.imread(sampleImage)
    resized_image = cv2.resize(bgr_img, dsize = (256, 256), interpolation = cv2.INTER_LINEAR)
    rgb_img = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
    faces = dlib.full_object_detections()
    dets = detector(rgb_img, 1)
    for det in dets:
        faces.append(sp(rgb_img, det))
        
    images = dlib.get_face_chips(rgb_img, faces, size = 256)
    count = 0
    for image in images:
        cv_rgb_image = np.array(image).astype(np.uint8)
        cv_bgr_image = cv2.cvtColor(cv_rgb_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite("align/sampleImage" + str(count) + ".jpg", cv_bgr_image)
        count = count + 1

In [81]:
getAlignment()

In [11]:
def default_loader(path):
    try:
        img = Image.open(path)
        return img.convert('RGB')
    except:
        print("Can not open {0}".format(path))
        
class myDataset(torch.utils.data.DataLoader):
    def __init__(self, img_dir, transform = None,loader = default_loader):
        img_list = []
        img_labels = []
        img_list.append(img_dir + '/1.jpg')
        img_labels.append(0)
                
        self.imgs = img_list
        self.labels = img_labels
        self.transform = transform 
        self.loader = loader 
        
    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self,index):
        img_path = self.imgs[index]
        label = torch.from_numpy(np.array(self.labels[index],dtype=np.int64))
        img = self.loader(img_path)
        if self.transform is not None:
            try:
                img = self.transform(img)
            except:
                print('Cannot transform image: {}'.format(img_path))
        return img,label
    
def getAge():
    model_dir = 'age-recognition/model_state.pth.tar'
    testdir = os.path.abspath('.') + '/age-recognition/test'

    model = models.__dict__['resnet18'](num_classes = 100)
    model = torch.nn.DataParallel(model).cuda()
    model.load_state_dict(torch.load(model_dir))

    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])

    test_loader = torch.utils.data.DataLoader(
            myDataset(testdir, transforms.Compose([
                #transforms.Resize(256),
                #transforms.RandomCrop((1500, 1000)),
                transforms.ToTensor(),
                normalize,
            ])),
            batch_size = 64, shuffle = False,
            num_workers = 4, pin_memory = True)

    criterion = nn.CrossEntropyLoss().cuda()
    # switch to evaluate mode  
    model.eval()

    with torch.no_grad():

        count = 0

        for i, (images, target) in enumerate(test_loader):
            target = target.cuda(None, non_blocking = True)            

            output = model(images)
            for j in range(len(output)):   
                print('prediction: ' + str((output[j].tolist()).index(max(output[j]))))

In [104]:
getAge()

prediction: 52


In [12]:
def getGender():
    model_dir = 'gender-recognition/model_state.pth.tar'
    testdir = os.path.abspath('.') + '/gender-recognition/test'

    model = models.__dict__['resnet18'](num_classes = 2)
    model = torch.nn.DataParallel(model).cuda()
    model.load_state_dict(torch.load(model_dir))

    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])

    test_loader = torch.utils.data.DataLoader(
            myDataset(testdir, transforms.Compose([
                #transforms.Resize(256),
                #transforms.RandomCrop((1500, 1000)),
                transforms.ToTensor(),
                normalize,
            ])),
            batch_size = 64, shuffle = False,
            num_workers = 4, pin_memory = True)

    criterion = nn.CrossEntropyLoss().cuda()
    # switch to evaluate mode  
    model.eval()

    with torch.no_grad():

        count = 0

        for i, (images, target) in enumerate(test_loader):
            target = target.cuda(None, non_blocking = True)            

            output = model(images)
            for j in range(len(output)):
                if (output[j].tolist()).index(max(output[j])) == 0:
                    print("male")
                else:
                    print("female")

In [13]:
getGender()

female


In [ ]:
# expression detection
# python ./Facial-Expression-Recognition.Pytorch-master/visualize.py
# the result of sample image is sad
# by github.com/WuJie1010/Facial-Expression-Recognition.Pytorch